In [30]:
!pip install llama-index-embeddings-openai
!pip install llama-index-vector-stores-pinecone
!pip install llama-index-llms-openai
!pip install llama-index

  Using cached pinecone-6.0.2-py3-none-any.whl.metadata (9.0 kB)
Using cached pinecone-6.0.2-py3-none-any.whl (421 kB)
  Attempting uninstall: pinecone
    Found existing installation: pinecone 7.0.2
    Uninstalling pinecone-7.0.2:
      Successfully uninstalled pinecone-7.0.2


In [9]:
!pip install python-dotenv pinecone llama-index pymupdf

In [5]:
from pinecone import Pinecone, Index, ServerlessSpec
import os

In [6]:
api_key = os.environ["pinecone_api_key"]
pc = Pinecone(api_key=api_key)

In [7]:
index_name = "llamaindex-rag-fs"

In [8]:
# dimensions are for text-embedding-ada-002
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=1536,
        metric="euclidean",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [9]:
pinecone_index = pc.Index(index_name)

In [11]:
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [12]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [15]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

--2025-06-08 01:02:00--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2025-06-08 01:02:00--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  65.9MB/s    in 0.2s    

2025-06-08 01:02:00 (65.9 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [16]:
import fitz

In [17]:
file_path = "./data/llama2.pdf"
doc = fitz.open(file_path)

In [19]:
from llama_index.core.node_parser import SentenceSplitter
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [ ]:
text_chunks = []

doc_idxs = []
for doc_idx, page in enumerate(doc)
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    docs_idxs.extend([doc_idx] * len(cur_text_chunks))